In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from ipywidgets import *
from matplotlib.animation import FuncAnimation
import matplotlib.animation as ani
from sloppy.raytracing import *
from sloppy.abcd import *

## Cavity geometry definition

In [3]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0, eps=np.zeros((6,5))):

    p5 = np.array([0,0,0])
    p6 = np.array([dx, dy, dzF])
    p1 = np.array([0, dy, dzF])
    p2 = np.array([dx, 2*dy, 0])
    p3 = np.array([d, dy+d, dz])
    p4 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    #add perturbations
    ps += eps[:,:3]
    axx = geom['xin']
    axy = 0.5*(geom['yin']+geom['yout'])
    ns = [rot_vec(rot_vec(ns[i], axx[i], np.deg2rad(eps[i,3])), axy[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]
    
    hi = 12.7
    qi=7.75
    #reference plane is expected between first and last element!
    elements = [CurvedMirror(p=ps[0], n=ns[0], diam=qi, R=Rfast, curv='CC', thet=angles[0]),\
                Mirror(p=ps[1], n=ns[1], diam=qi),\
                Mirror(p=ps[2], n=ns[2], diam=hi),\
                Mirror(p=ps[3], n=ns[3], diam=hi),\
                Mirror(p=ps[4], n=ns[4], diam=qi),\
                CurvedMirror(p=ps[5], n=ns[5], diam=qi, R=Rfast, curv='CC', thet=angles[5])]
    
    M = geom['M']
    R = geom['R']
    abcd = []
    for i, el in enumerate(elements):
        d = np.linalg.norm(M[i-1,i])/2
        #Mrt = Prop(d)@Rot(R[i])@Mirror(roc[i], angles[i])@Prop(d)@Mrt
        abcd.extend([Prop(d), el, Rot(R[i]), Prop(d)])
      
    #add screen to elements
    x0 = 0.5*(elements[0].p + elements[-1].p)
    n0 = norm(elements[0].p - elements[-1].p)

    screen = Screen(p=x0, n=-n0, diam=qi)
    elements.append(screen)

    return elements, abcd, geom

## Paraxial modes and degeneracy

In [4]:
stab = lambda m: abs(0.5*np.trace(m))<1
La = 27.77

scanrng = 1.
N = 300
Las = La + np.linspace(-scanrng, scanrng, N)

ms = np.zeros((N))
ws = np.zeros((N,2))
freqs = np.zeros((N,4))
for i, l in enumerate(Las):
    _, abcd, _ = SixMirror(dx=l)
    system = ABCDSystem(abcd)
    
    ms[i] = stab(system.abcd_rt)
    w = system.waist_at(0)
    ws[i,:] = np.sort(w)
    freqs[i,...] = np.concatenate(system.get_freqs())
    
g, ax = plt.subplots(ncols=2, figsize=(8,4))
ax[0].plot(Las, ws*1e3)
ax[0].set_ylabel('um')
ax[1].plot(Las, freqs*1e-6)
ax[1].set_ylabel('MHz')
ax[1].axhline(0, color='grey')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Paraxial eigenrays and hit pattern

In [5]:
elements, abcd, geom = SixMirror(dx=27.2952)
system = ABCDSystem(abcd)
k = 2*np.pi/system.wl
mu1, mu2 = system.q

mu = mu1.real*0.05

In [17]:
def makeidx(hit_m):
    return np.arange(hit_m.shape[0])

In [18]:
hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=500)

plt.figure()
line = plt.scatter(hit_m[:,0], hit_m[:,1], c=makeidx(hit_m), cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Raytracing

In [7]:
#turn the paraxial eigenrays (eigenvectors of ABCD matrix) into rays (position and slope three-vectors for raytracing)
x0 = elements[-1].p #elementts[-1] is the screen
n0 = elements[-1].n
ray0 = eigenvector_to_ray(mu, x0, k)

In [8]:
traj0 = propagate_system(elements, ray0, Nrt=30, clip=True)

In [11]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)

plot += k3d.vectors(origins=ray0[0,...], vectors=ray0[1,...], colors=[(0xff0000,)*2], head_size=2.)
plot_trajs(traj0, plot, shader='mesh', width=0.2, color=0x00ff00)

plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [19]:
traj_hit = propagate_system_at(elements, ray0, which=6, Nrt=500)[:,0,:,:] #only positions
hit_scr = traj_hit[:,0,:]

plt.figure()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=makeidx(hit_scr), cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
abcd_fd = extract_ABCD_fd(elements, x0, n0, epsr=1e-3, epss=1e-3) 

In [15]:
#check that the returned matrix is symplectic
G = np.array([[0,0,1,0], [0,0,0,1], [-1,0,0,0], [0,-1,0,0]])
np.round(abcd_fd.T@G@abcd_fd, decimals=3)

array([[-0.   ,  0.   ,  1.   ,  0.   ],
       [-0.   ,  0.   , -0.   ,  1.   ],
       [-1.   ,  0.   , -0.   , -0.001],
       [-0.   , -1.   ,  0.001,  0.   ]])

## Main method comparison

In [31]:
fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
ms = 8.
linea = ax.scatter(hit_scr[:,1]-x0[1], hit_scr[:,2]-x0[2], c=makeidx(hit_scr), cmap='jet', s=ms)
linec = cx.scatter(hit_scr[:,1]-x0[1], hit_scr[:,2]-x0[2], c=makeidx(hit_scr), cmap='jet', s=ms)
lineb = bx.scatter(hit_m[:,0], hit_m[:,1], c=makeidx(hit_m), cmap='jet', s=ms)
ax.set_title('Raytracing')
bx.set_title('ABCD')
cx.set_title('ABCD (Extracted)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')


def update(dl=27.295, ar=0.01, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements, abcd, _ = SixMirror(dx=dl)#27.2952+
        x0 = elements[-1].p #elementts[-1] is the screen
        n0 = elements[-1].n
        system = ABCDSystem(abcd)
        abcd_fd = extract_ABCD_fd(elements, x0, n0, epsr=1e-3, epss=1e-3) 

        mu1, mu2 = system.q
        mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        ray0 = eigenvector_to_ray(mu, x0, k)
 
        traj_hit = propagate_system_at(elements, ray0, which=6, Nrt=Nrt)[:,0,:,:] #only get positions
        hit_scr = traj_hit[:,0,:]-x0

        linea.set_offsets(hit_scr[:,1:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()
        
interactive(update, dl=(27.2, 27.9, 1e-2), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))
#interactive(update, dl=(27.75, 27.81, 1e-3), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=27.295, description='dl', max=27.9, min=27.2, step=0.01), FloatSlider(…

In [32]:
fig.savefig('logo.png', dpi=600)
fig.savefig('logo.svg', dpi=600)